<a href="https://colab.research.google.com/github/LC1332/Embed-Adapter/blob/main/notebook/100_setences_testing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [ ] 准备100个text的数据
- [ ] 抽取各种embedding
- [ ] 载入adapter
- [ ] 1nn搜索验证


In [1]:
%cd /content
!rm -rf /content/Embed-Adapter
!git clone https://github.com/LC1332/Embed-Adapter
%cd /content/Embed-Adapter

/content
Cloning into 'Embed-Adapter'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 127 (delta 3), reused 6 (delta 3), pack-reused 115
Receiving objects: 100% (127/127), 35.06 MiB | 15.75 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/Embed-Adapter


In [2]:
from EmbedAdapter.utils import get_general_embedding

构造100条数据

In [3]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
fname = "/content/drive/MyDrive/unpublic/zhwiki_2k_embedding.jsonl"

import json

raw_datas = []

with open(fname, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip() == "":
            continue
        data = json.loads(line)
        raw_datas.append(data)

In [5]:
datas = []
for data in raw_datas:
    datas.append({
        "text": data["left"],
        "embedding": data["left_embedding"],
    })
    datas.append({
        "text": data["right"],
        "embedding": data["right_embedding"],
    })


对于datas中的数据，我希望通过text的长度进行降序

请用python为我实现

In [6]:
sorted_datas = sorted(datas, key=lambda x: len(x["text"]), reverse=True)

In [7]:
print(len(sorted_datas[0]['text']))

2562


In [8]:
test_texts = []
embed_openai = []
record_text_prefix = set()

for i in range(200):
    current_prefix = sorted_datas[i]['text'][:50]
    if current_prefix in record_text_prefix:
        continue
    record_text_prefix.add(current_prefix)
    test_texts.append(sorted_datas[i]['text'])
    embed_openai.append(sorted_datas[i]['embedding'])

    if len(test_texts) == 100:
        break

In [9]:
model2embed = {}
model2embed['openai'] = embed_openai

In [10]:
from EmbedAdapter.utils import get_general_embedding

In [12]:
model_names = ["BAAI/bge-large-zh-v1.5", "BAAI/bge-small-zh-v1.5", "BAAI/bge-base-zh-v1.5"]

In [14]:
for model_name in model_names:
    model2embed[model_name] = get_general_embedding(test_texts, model_name)

  0%|          | 0/7 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/409M [00:00<?, ?B/s]

100%|██████████| 7/7 [00:19<00:00,  2.83s/it]


In [15]:
test_data = {
    "texts": test_texts,
    "model2embed": model2embed,
}

In [16]:
import pickle

# Save corr_map to a pickle file
with open('/content/test_data.pkl', 'wb') as f:
    pickle.dump(test_data, f)

In [17]:
print(model2embed["BAAI/bge-large-zh-v1.5"][0])

[-0.020646652206778526, -0.00742915878072381, -0.012680284678936005, 0.014829138293862343, 0.010811807587742805, -0.0030179645400494337, 0.0018011739011853933, -0.035392049700021744, 0.031411122530698776, 0.013580499216914177, -0.01926445960998535, -0.02913973107933998, -0.02977304346859455, 0.017638854682445526, -0.06814275681972504, 0.015945930033922195, -0.013336045667529106, -0.025982515886425972, 0.011097957380115986, 0.002050875686109066, 0.04522227123379707, 0.04590708017349243, 0.047193605452775955, -0.019527731463313103, 0.019856536760926247, -0.06745709478855133, -0.03003641776740551, -0.006290256977081299, 0.001075371983461082, -0.01927340403199196, -0.0015016444958746433, -0.009140589274466038, 0.001214831368997693, 0.006638381164520979, -0.007805719505995512, 0.05282853916287422, -0.026311447843909264, 0.0086510656401515, 0.00377241731621325, -0.04365774244070053, -0.0005687996163032949, 0.0279773510992527, 0.00524698942899704, -0.004221365321427584, 0.02864721231162548, -